In [1]:
import svfpy
import rasterio
import numpy as np

In [2]:
# np.concatenate([np.zeros((10000, 10000, 6)), np.ones((10000, 10000, 3))], axis=2).shape

In [3]:
mds = '../test/data/mds_sao_paulo_city_4000.tiff'

In [4]:
svf = svfpy.SVF(mds_path=mds, kernel_size=11)

In [5]:
profile = svf.mds_src.profile

In [6]:
angles = svfpy.calculate(svf)

3.0
.. difff
 .. angles


In [8]:
max_angles = np.zeros((4000,4000,40), dtype=np.float16)

In [9]:
masks = svfpy.lines_mask(svf)

In [35]:
#### TODO
## Ta quase
np.argwhere(masks).reshape(40,6,3)[:,:,1:]
np.max(angles[:, :, [[0,3,5,6], [0,1,2,3,]], [[0,4,5,6], [0,1,2,3]]], axis=2)


array([[[ 0.1405,  0.    ,  0.    ,  0.    ],
        [ 0.1405,  0.    ,  0.    ,  0.    ],
        [ 0.    ,  0.    ,  0.    ,  0.    ],
        ...,
        [-0.2756,  0.    ,  0.    , -0.3398],
        [ 0.2756,  0.3398,  0.2314,  0.3398],
        [ 0.4014,  0.4875,  0.2314,  0.    ]],

       [[ 0.1405,  0.    ,  0.    ,  0.    ],
        [ 0.    ,  0.    ,  0.    ,  0.    ],
        [ 0.    ,  0.    ,  0.    ,  0.    ],
        ...,
        [ 0.1405,  0.4204,  0.2314,  0.3398],
        [ 0.4014,  0.7295,  0.4407,  0.815 ],
        [ 0.4014,  0.3398,  0.4407,  0.6157]],

       [[ 0.    ,  0.    ,  0.    ,  0.    ],
        [ 0.    ,  0.    ,  0.    ,  0.    ],
        [ 0.    ,  0.    ,  0.    ,  0.    ],
        ...,
        [ 0.4014,  0.93  ,  0.6157,  0.815 ],
        [ 0.4014,  0.93  ,  0.6157,  0.815 ],
        [ 0.4014,  0.4204,  0.6157,  0.815 ]],

       ...,

       [[ 0.    ,  0.    ,  0.    ,  0.    ],
        [ 0.    ,  0.    ,  0.    ,  0.    ],
        [ 0.    ,  0. 

In [ ]:
for k, m in enumerate(mdss):
    transform = transformations[k]
    profile.update(
        transform=transform,
        width=m.shape[0],
        height=m.shape[1]
    )

    with rasterio.open(f'../tmp/raw-{k}-{m.shape[1]}.tif', 'w', **profile) as dst:
        dst.write(m, 1)